<a href="https://colab.research.google.com/github/SEDHUMADHAVAN-CYBER/Multi-Agentx/blob/main/Multi_Agent_Langgraph_Agentx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
!pip install -q langgraph langchain-groq


In [70]:
from typing import TypedDict, List, Optional


In [71]:
class AgentState(TypedDict, total=False):
    user_input: str

    plan: List[str]

    collaborator_output: Optional[str]
    competitor_output: Optional[str]

    negotiated_output: Optional[str]   # only used if negotiation agent added

    final_output: Optional[str]
    reward: Optional[str]

    reward_score: Optional[int]        # numeric reward


In [72]:
from google.colab import userdata
from langchain_groq import ChatGroq

llm = ChatGroq(
    groq_api_key=userdata.get("GROQ_API_KEY"),
    model_name="llama-3.1-8b-instant"
)


In [73]:
memory = []

def memory_agent(state: AgentState) -> AgentState:
    global memory
    entry = f"Task: {state['user_input']} | Reward: {state.get('reward')}"
    memory.append(entry)
    return {"history": memory}


In [74]:
def planner_agent(state: AgentState) -> AgentState:
    prompt = f"Break this task into 3 clear steps:\n{state['user_input']}"
    response = llm.invoke(prompt)

    steps = [
        s.strip("- ").strip()
        for s in response.content.split("\n")
        if s.strip()
    ]

    if not steps:
        steps = ["Analyze", "Explain", "Conclude"]

    print("🧠 Planner:", steps)
    return {"plan": steps}


In [75]:
def collaborator_agent(state: AgentState) -> AgentState:
    prompt = f"""
    Task: {state['user_input']}
    Plan: {state['plan']}

    Provide a helpful, optimistic solution.
    """
    response = llm.invoke(prompt)

    print("🤝 Collaborator responding")
    return {"collaborator_output": response.content}


In [76]:
def competitor_agent(state: AgentState) -> AgentState:
    prompt = f"""
    Task: {state['user_input']}

    Critique the collaborator's approach.
    Provide a different perspective.
    """
    response = llm.invoke(prompt)

    print("⚔️ Competitor responding")

    return {"competitor_output": response.content}


In [77]:
import random

def judge_agent(state: AgentState) -> AgentState:
    prompt = f"""
    Task: {state['user_input']}

    Collaborator Answer:
    {state['collaborator_output']}

    Competitor Answer:
    {state['competitor_output']}

    Decide which answer is better and explain briefly.
    """
    response = llm.invoke(prompt)

    score = random.randint(1, 10)

    return {
        "final_output": response.content,
        "reward": f"Reward Score = {score}",
        "reward_score": score
    }


In [78]:
def negotiation_agent(state: AgentState) -> AgentState:
    prompt = f"""
    Collaborator said:
    {state['collaborator_output']}

    Competitor said:
    {state['competitor_output']}

    Combine both answers into a stronger final answer.
    """
    response = llm.invoke(prompt)

    return {"negotiated_output": response.content}


In [79]:
def train_until_good_answer(question, threshold=8):
    while True:
        final_state = multi_agent_graph.invoke({"user_input": question})
        score = final_state.get("reward_score", 0)

        print("Reward:", score)

        if score >= threshold:
            print("🎉 Good answer achieved!")
            return final_state


In [80]:
from langgraph.graph import StateGraph

graph = StateGraph(AgentState)

graph.add_node("planner", planner_agent)
graph.add_node("collaborator", collaborator_agent)
graph.add_node("competitor", competitor_agent)
graph.add_node("judge", judge_agent)

graph.set_entry_point("planner")

graph.add_edge("planner", "collaborator")
graph.add_edge("planner", "competitor")

graph.add_edge("collaborator", "judge")
graph.add_edge("competitor", "judge")
graph.add_node("memory", memory_agent)
graph.add_edge("judge", "memory")
graph.set_finish_point("memory")


graph.set_finish_point("judge")
graph.add_node("negotiator", negotiation_agent)

graph.add_edge("collaborator", "negotiator")
graph.add_edge("competitor", "negotiator")

graph.add_edge("negotiator", "judge")


# ✅ COMPILE HERE — THIS IS THE CORRECT BLOCK
multi_agent_graph = graph.compile()


In [81]:
state = {"user_input": "what is mean by anesthesia"}

final_state = multi_agent_graph.invoke(state)

print(final_state)


🧠 Planner: ['Here are 3 clear steps explaining what anesthesia means:', '**Step 1: Understanding Anesthesia**', 'Anesthesia refers to a medical treatment that helps reduce or eliminate a person\'s feeling of pain, discomfort, or sensation during medical procedures, surgeries, or other medical interventions. The word "anesthesia" comes from the Greek words "a-" (meaning "without" or "absence of") and "næsthesia" (meaning "sensation" or "feeling").', '**Step 2: Types of Anesthesia**', 'There are several types of anesthesia, including:', '**Local anesthesia**: numbs a specific area of the body, such as a tooth or a limb.', '**Regional anesthesia**: numbs a larger area of the body, such as an arm or leg.', '**General anesthesia**: induces a state of unconsciousness, so the person feels no pain or discomfort.', '**Step 3: Purpose of Anesthesia**', 'The primary purpose of anesthesia is to reduce or eliminate pain and discomfort during medical procedures, allowing patients to feel more relaxe